In [3]:
#use tr
import torch
import os
import gc
import json
# 设置 PYTORCH_CUDA_ALLOC_CONF 以避免内存碎片化
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
torch.cuda.empty_cache()
gc.collect()

0

In [5]:
def prepare_training_data(json_file, output_file):
    """将 JSON 数据转换为训练用的纯文本格式"""
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(output_file, "w", encoding="utf-8") as f:
        for entry in data:
            # 提取问题部分，并去除多余空格和换行符
            question = entry["content"].replace("\n", " ").strip()
            
            # 连接所有回复为一个字符串，过滤空回复
            responses = [resp.strip() for resp in entry["reponses"] if resp.strip()]
            answer = " ".join(responses)

            # 如果问题或答案为空，则跳过该条数据
            if not question or not answer:
                continue

            # 写入文件
            f.write(f"Q: {question}\nA: {answer}\n\n")

# 将 JSON 数据转换为文本格式供训练使用
prepare_training_data("./ptt_articles.json", "train.txt")
print("数据已成功转换并保存为 train.txt")

数据已成功转换并保存为 train.txt


In [6]:

torch.cuda.empty_cache()
gc.collect()


0

In [7]:
'''
import torch
from unsloth import FastLanguageModel
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, TextDataset

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载 LLaMA-1B 模型并启用 4-bit 和 FP16
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",
    device_map={"": "cuda"},       # 将模型所有模块加载到 GPU 上
    dtype=torch.float16,      # 使用半精度 FP16，减少显存占用
    max_length=30,
    load_in_4bit=True,             # 启用 4-bit 量化
)
model.to(device)

# 加载训练数据集
def load_dataset(file_path, block_size=512):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

train_dataset = load_dataset("train.txt")

# 创建数据批处理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # LLaMA 是自回归模型
)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./finetuned_llama",
    overwrite_output_dir=True,
    num_train_epochs=3,                  # 训练轮数
    per_device_train_batch_size=1,       # 小批次，防止 OOM
    gradient_accumulation_steps=8,       # 梯度累积，模拟更大批次
    learning_rate=5e-5,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# 开始微调
print("开始微调模型...")
trainer.train()
print("模型微调完成并已保存！")

# 保存微调后的模型和分词器
trainer.save_model("./finetuned_llama")
tokenizer.save_pretrained("./finetuned_llama")
'''

'\nimport torch\nfrom unsloth import FastLanguageModel\nfrom transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, TextDataset\n\n# 检查 GPU 是否可用\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n# 加载 LLaMA-1B 模型并启用 4-bit 和 FP16\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    model_name="unsloth/Llama-3.2-1B-Instruct",\n    device_map={"": "cuda"},       # 将模型所有模块加载到 GPU 上\n    dtype=torch.float16,      # 使用半精度 FP16，减少显存占用\n    max_length=30,\n    load_in_4bit=True,             # 启用 4-bit 量化\n)\nmodel.to(device)\n\n# 加载训练数据集\ndef load_dataset(file_path, block_size=512):\n    return TextDataset(\n        tokenizer=tokenizer,\n        file_path=file_path,\n        block_size=block_size\n    )\n\ntrain_dataset = load_dataset("train.txt")\n\n# 创建数据批处理器\ndata_collator = DataCollatorForLanguageModeling(\n    tokenizer=tokenizer, mlm=False  # LLaMA 是自回归模型\n)\n\n# 设置训练参数\ntraining_args = TrainingArguments(\n    output_dir="./finetune

In [9]:
from transformers import TextDataset, DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, BertTokenizer
from transformers import GPT2TokenizerFast

# 使用 GPT2 的快速分词器，适用于中文
model_name = "uer/gpt2-chinese-cluecorpussmall"

# 加载 GPT-2 模型和分词器
tokenizer = BertTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
#tokenizer = GPT2Tokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
#tokenizer(return_tensors="pt", truncation=True, max_length=512)
# 加载 GPT-2 分词器
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # 设置 EOS 为 padding token

def load_dataset(file_path, block_size=512):
    """加载文本数据集并进行分块处理"""
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size  # 每个样本的最大长度
    )

train_dataset = load_dataset("train.txt")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # GPT-2 不使用 MLM 任务
)


vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

/home/kaia/miniforge3/envs/TR/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [10]:
torch.cuda.empty_cache()
gc.collect()

328

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned_gpt2",   # 模型保存路径
    overwrite_output_dir=True,       # 覆盖现有文件
    num_train_epochs=3,              # 训练轮数
    per_device_train_batch_size=1,   # 每个设备的批大小
    gradient_accumulation_steps=8,   # 梯度累积
    learning_rate=5e-5,              # 学习率
    logging_dir="./logs",            # 日志路径
    logging_steps=100,               # 每100步记录一次日志
    save_steps=500,                  # 每500步保存一次模型
    save_total_limit=2,              # 最多保存2个检查点
)


In [12]:
torch.cuda.empty_cache()
gc.collect()

20

In [13]:
from transformers import GPT2LMHeadModel, Trainer

# 加载预训练的 GPT-2 模型
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)


In [14]:
torch.cuda.empty_cache()
gc.collect()
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [15]:
print("开始微调模型...")
trainer.train()
print("微调完成！")

# 保存微调后的模型和分词器
trainer.save_model("./finetuned_gpt2")
tokenizer.save_pretrained("./finetuned_gpt2")


开始微调模型...


  0%|          | 0/21 [00:00<?, ?it/s]

{'train_runtime': 10.9012, 'train_samples_per_second': 17.062, 'train_steps_per_second': 1.926, 'train_loss': 3.4956483386811756, 'epoch': 2.71}
微调完成！


('./finetuned_gpt2/tokenizer_config.json',
 './finetuned_gpt2/special_tokens_map.json',
 './finetuned_gpt2/vocab.txt',
 './finetuned_gpt2/added_tokens.json')

In [16]:
torch.cuda.empty_cache()
gc.collect()

58

In [18]:
# 加载预训练模型和分词器
original_model = GPT2LMHeadModel.from_pretrained("gpt2")
original_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 加载微调后的模型和分词器
finetuned_model = GPT2LMHeadModel.from_pretrained("./finetuned_gpt2")
finetuned_tokenizer = BertTokenizer.from_pretrained("./finetuned_gpt2")

# 将模型移动到 GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
original_model.to(device)
finetuned_model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=21128, bias=False)
)

In [19]:
torch.cuda.empty_cache()
gc.collect()


34

In [53]:
def generate_response(model, tokenizer, prompt):##, max_length=256
    """生成文本的函数"""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    #input_ids = inputs['input_ids']
    #attention_mask = input['attention_mask']
    #print(inputs)
    #print(inputs["input_ids"][0])
    with torch.no_grad():
        '''
        output = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,      # 使用随机采样生成更有趣的结果
            top_k=50,            # 限制最高 k 个候选词
            top_p=0.95           # 使用 nucleus sampling 生成
        )
        '''
        output = model.generate(
        **inputs,
        do_sample=True,
        temperature=0.7,  # 控制生成的隨機性
        max_new_tokens=50,  # 控制生成長度
        top_p=0.9,  # 使用 nucleus sampling
        top_k=50,  # 考慮前 50 個 token
        repetition_penalty=2.0,
        pad_token_id=tokenizer.eos_token_id,  # 填充 token
        eos_token_id=tokenizer.eos_token_id,  # 結束 token
        num_return_sequences=1
        #clean_up_tokenization_spaces=True   #if use generator in pipeline or mayber gpt-4 than set the value
        )
    # 确保 output 是整数列表
    token_ids = output[0].tolist()
    decoded_text = tokenizer.decode(token_ids, skip_special_tokens=True)
    #return tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_text

In [54]:
torch.cuda.empty_cache()
gc.collect()


682

In [56]:
from transformers import GPT2TokenizerFast

# 定义测试提示
prompt = "請問有人吃過樂復得來治療強迫症嗎 或者因其它的精神官能症吃樂復得有什麼副作用嗎 可以跟我告知你或你朋友的副作用是什麼嗎 謝謝"

# 微调前的输出
print("微调前的输出：")
original_output = generate_response(original_model, original_tokenizer, prompt)
print(f'original_output:{original_output}')
#print(original_output)
'''
for i in range(len(original_output)):##沒有一次generate的嗎?
    #print(output[i].shape)
    #generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
    generated_text = tokenizer.decode(original_output[i], skip_special_tokens=True)
    # on purpose of to make sure that a sentences end in some certain  Marks.
    if not generated_text[-1] in ".!?":
        last_punctuation = max(generated_text.rfind(p) for p in ".!?")
        if last_punctuation != -1:
            generated_text = generated_text[:last_punctuation + 1]
    print(f"{i} sentences: {generated_text}")
'''
# 微调后的输出
print("\n微调后的输出：")
finetuned_output = generate_response(finetuned_model, finetuned_tokenizer, prompt)
print(f'finetuned_output:{finetuned_output}')
'''
for i in range(len(finetuned_output)):##沒有一次generate的嗎?
    #print(output[i].shape)
    #generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
    generated_text = tokenizer.decode(finetuned_output[i], skip_special_tokens=True)
    # on purpose of to make sure that a sentences end in some certain  Marks.
    if not generated_text[-1] in ".!?":
        last_punctuation = max(generated_text.rfind(p) for p in ".!?")
        if last_punctuation != -1:
            generated_text = generated_text[:last_punctuation + 1]
    print(f"{i} sentences: {generated_text}")
'''



微调前的输出：
original_output:請問有人吃過樂復得來治療強迫症嗎 或者因其它的精神官能症吃樂復得有什麼副作用嗎 可以跟我告知你或你朋友的副作用是什麼嗎 謝謝十五学务交大多就说差不教对上行,耳圣王相俢必親歐一舞。
The fact that

微调后的输出：
finetuned_output:請 問 有 人 吃 過 樂 復 得 來 治 療 強 迫 症 嗎 或 者 因 其 它 的 精 神 官 能 症 吃 樂 復 得 有 什 麼 副 作 用 嗎 可 以 跟 我 告 知 你 或 你 朋 友 的 副 作 用 是 什 麼 嗎 謝 謝 ， 您 好 。 沒 事 說 話 多 了 就 會 被 罵 要 不 然 還 想 讓 別 給 他 們 點 個 讚. 這 樣 : 對 於 何 處 ？ 無 論 聽 一 般 都 覺 著 ： 比 較 少 數


'\nfor i in range(len(finetuned_output)):##沒有一次generate的嗎?\n    #print(output[i].shape)\n    #generated_text = tokenizer.decode(output[i], skip_special_tokens=True)\n    generated_text = tokenizer.decode(finetuned_output[i], skip_special_tokens=True)\n    # on purpose of to make sure that a sentences end in some certain  Marks.\n    if not generated_text[-1] in ".!?":\n        last_punctuation = max(generated_text.rfind(p) for p in ".!?")\n        if last_punctuation != -1:\n            generated_text = generated_text[:last_punctuation + 1]\n    print(f"{i} sentences: {generated_text}")\n'